In [1]:
!pip install scikit-learn==1.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.21.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.0.2 which is incompatible.


In [2]:
!pip freeze

absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.3
aiohttp==3.10.9
aiosignal==1.3.1
alabaster==0.7.16
albucore==0.0.16
albumentations==1.4.15
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.19.0
astropy==6.1.4
astropy-iers-data==0.2024.10.7.0.32.46
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.21.0
bigquery-magics==0.4.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.4.3
Bottleneck==1.4.0
bqplot==0.12.43
branca==0.8.0
build==1.2.2.post1
CacheControl==0.14.0
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.87
clarabel==0.9.0
click==8.1.7
cloudpathlib==0.19.0
cloudpickle==2.2.1
cmake==3.30.4
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==

# Pipeline

In [3]:
#Instalaciones
import nltk

import tabulate
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords

import re
from typing_extensions import *
from typing import *

import numpy as np
import pandas as pd
import re
import plotly.express as px


from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('stopwords')
nltk.download('wordnet')

import string

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


# 1. Lectura de datos

In [4]:
GITHUB_URL: Final[str] = 'https://raw.githubusercontent.com/LoganTaurus/DB-Project-01/main/'

## Carga de datos de Entrenamiento

In [5]:
# Leer el archivo XLSX
df = pd.read_excel(GITHUB_URL + 'ODScat-345.xlsx')

# Si hay problemas de codificación, puedes intentar corregirlos
df = df.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)

<ipython-input-5-9f84655360b9>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)


## Carga de datos a predecir

In [6]:
# Leer el archivo XLSX
df_predecir = pd.read_excel(GITHUB_URL + 'TestODScat-345.xlsx')

# Si hay problemas de codificación, puedes intentar corregirlos
df_predecir = df_predecir.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)

<ipython-input-6-7296460dce77>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_predecir = df_predecir.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)


# 2. Construcción del pipeline

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import joblib

# Clase personalizada para preprocesar los datos
class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stopwords = set(stopwords.words('spanish'))
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cleaned_data = []
        for text in X:
            text = re.sub(r'\d+', '', text)  # Remove digits
            text = ' '.join([self.lemmatizer.lemmatize(word) for word in text.split() if word not in self.stopwords])
            cleaned_data.append(text)
        return cleaned_data

# Función para entrenar el modelo
def entrenar_modelo(df):
    X = df['Textos_espanol']
    y = df['sdg']

    # Codificar etiquetas
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)

    # Pipeline para preprocesar, vectorizar y clasificar
    pipeline = Pipeline([
        ('preprocessor', CustomPreprocessor()),
        ('tfidf', TfidfVectorizer(max_features=3000)),
        ('model', MultinomialNB())
    ])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    pipeline.fit(X_train, y_train)

    # Guardar modelo y el encoder
    joblib.dump(pipeline, 'ModeloODS.joblib')
    joblib.dump(encoder, 'Encoder.joblib')

    # Métricas de desempeño
    y_pred = pipeline.predict(X_test)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Probabilidades de las predicciones
    y_pred_proba = pipeline.predict_proba(X_test)  # Predicciones con probabilidad
    y_pred_probabilidades = y_pred_proba.max(axis=1)  # Probabilidad máxima de la clase predicha

    # Redondear las probabilidades a 2 decimales
    y_pred_probabilidades = pd.Series(y_pred_probabilidades).round(2)

    # Agregar las probabilidades al DataFrame
    df_predicciones = pd.DataFrame({'Predicción': y_pred, 'Probabilidad': y_pred_probabilidades})

    return precision, recall, f1, df_predicciones


In [8]:
entrenar_modelo(df)

(0.9788842000055292,
 0.9786008230452675,
 0.9786078662482677,
       Predicción  Probabilidad
 0              1          0.96
 1              0          0.92
 2              0          0.88
 3              0          0.99
 4              2          0.69
 ...          ...           ...
 1210           1          0.90
 1211           0          0.87
 1212           1          0.92
 1213           2          0.85
 1214           2          0.87
 
 [1215 rows x 2 columns])

# 3. Testear el funcionamineto del pipeline

In [9]:
import numpy as np
import pandas as pd
import joblib

# Cargar el pipeline y el encoder entrenados
pipeline = joblib.load('ModeloODS.joblib')

# Mapa inverso para convertir 0, 1, 2 de vuelta a 3, 4, 5 después de la predicción
reverse_label_map = {0: 3, 1: 4, 2: 5}

# Seleccionar la columna de textos en español
X_predict = df_predecir['Textos_espanol']

# Preprocesar los textos no etiquetados (utilizar el preprocesador del pipeline)
preprocessed_texts = pipeline.named_steps['preprocessor'].transform(X_predict)

# Convertir los textos preprocesados en vectores numéricos usando TfidfVectorizer
vectorized_texts = pipeline.named_steps['tfidf'].transform(preprocessed_texts)

# Realizar las predicciones utilizando el modelo entrenado
predictions = pipeline.named_steps['model'].predict(vectorized_texts)

# Obtener las probabilidades de las predicciones
predictions_proba = pipeline.named_steps['model'].predict_proba(vectorized_texts)

# Obtener la probabilidad más alta para cada predicción
predictions_probabilities = predictions_proba.max(axis=1)

# Redondear las probabilidades a dos decimales
predictions_probabilities = np.round(predictions_probabilities, 2)

# Convertir las predicciones de vuelta a 3, 4, 5
predictions_decoded = pd.Series(predictions).map(reverse_label_map)

# Agregar las predicciones decodificadas y sus probabilidades al DataFrame original de datos no etiquetados
df_predecir['sdg'] = predictions_decoded
df_predecir['probabilidad'] = predictions_probabilities

In [10]:
# Guardar el DataFrame con las predicciones en un nuevo archivo
df_predecir.to_csv('archivo_con_predicciones_no_etiquetados.csv', index=False)

print("Predicciones guardadas exitosamente en archivo_con_predicciones_no_etiquetados.csv")

Predicciones guardadas exitosamente en archivo_con_predicciones_no_etiquetados.csv


In [11]:
# Revisión de las primeras filas del DataFrame con las predicciones
df_predecir.head(100)

,Textos_espanol,sdg,probabilidad
0,Han examinado la contribución de las universi...,4,0.74
1,"En la ltima dcada, y en particular desde 2010,...",3,0.81
2,En qu pases los estudiantes de alto rendimient...,4,0.96
3,A raz de su preocupacin por el hecho de que lo...,3,0.97
4,"1999|H5|, Ares Abalde, 2014[ij]. El pequeo nme...",4,0.97
...,...,...,...
95,Algunos pases han adaptado estas preguntas a s...,5,0.90
96,"En la mayora de los pases, las polticas moneta...",5,0.62
97,El equipo de revisin de la OCDE qued impresion...,4,0.97
98,"Aqu nuevamente, el debate est abierto. Cabe me...",5,0.71
